##Internship Task


**Task Title: Comparative Study of Deep Learning Models on CIFAR Dataset**

Task Overview

Section 1: LeNet Model – Training & Evaluation on CIFAR

Section 2: ResNet Model – Training & Evaluation on CIFAR

Section 3: Transformer Model – Training & Evaluation on CIFAR

Section 4: VGG16 Model – Training & Evaluation on CIFAR

#Section 1: LeNet Model – Training & Evaluation on CIFAR

In [1]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(factor=0.5, patience=2),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

Import Libraries

In [2]:
import tensorflow as tf
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, AveragePooling2D, Flatten, Dense
from sklearn.metrics import precision_score, recall_score
import numpy as np

Load and Preprocess CIFAR -10

In [3]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0  # Normalize
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

170498071/170498071 ━━━━━━━━━━━━━━━━━━━━ 4s 0us/step


Define LeNet Architecture

In [4]:
model = Sequential([
    Conv2D(6, kernel_size=5, activation='relu', padding='same', input_shape=(32, 32, 3)),
    AveragePooling2D(pool_size=(2, 2)),
    Conv2D(16, kernel_size=5, activation='relu'),
    AveragePooling2D(pool_size=(2, 2)),
    Flatten(),
    Dense(120, activation='relu'),
    Dense(84, activation='relu'),
    Dense(10, activation='softmax')
])

/usr/local/lib/python3.11/dist-packages/keras/src/layers/convolutional/base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Compile and Train

In [5]:
from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks)

Epoch 1/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 8s 6ms/step - accuracy: 0.2682 - loss: 2.0009 - val_accuracy: 0.4224 - val_loss: 1.6080 - learning_rate: 5.0000e-04
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 2s 3ms/step - accuracy: 0.4396 - loss: 1.5501 - val_accuracy: 0.4521 - val_loss: 1.5549 - learning_rate: 5.0000e-04
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 0.4852 - loss: 1.4315 - val_accuracy: 0.5094 - val_loss: 1.3876 - learning_rate: 5.0000e-04
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 6s 5ms/step - accuracy: 0.5218 - loss: 1.3388 - val_accuracy: 0.5244 - val_loss: 1.3369 - learning_rate: 5.0000e-04
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5448 - loss: 1.2828 - val_accuracy: 0.5274 - val_loss: 1.3290 - learning_rate: 5.0000e-04
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 5s 4ms/step - accuracy: 0.5673 - loss: 1.2185 - val_accuracy: 0.5447 - val_loss: 1.2686 - learning_rate: 5.0000e-04
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 3s 4ms/step - accuracy: 

  Predict and Evaluate

In [6]:

y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test_cat, axis=1)

accuracy = np.mean(y_pred_labels == y_true_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='macro')
recall = recall_score(y_true_labels, y_pred_labels, average='macro')

print("LeNet on CIFAR-10")
print(f"Accuracy:  {accuracy:.4f}")
print(f"Precision: {precision:.4f}")
print(f"Recall:    {recall:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 6ms/step
LeNet on CIFAR-10
Accuracy:  0.6186
Precision: 0.6161
Recall:    0.6186


#Section 2: ResNet Model – Training & Evaluation on CIFAR

In [16]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(factor=0.5, patience=2),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

In [17]:
import tensorflow as tf
from tensorflow.keras.applications import ResNet50
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Input
from sklearn.metrics import precision_score, recall_score
import numpy as np

Load and Preprocess CIFAR-10

In [18]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

Define ResNet50 Model (without top, add custom head)

In [19]:

base_model = ResNet50(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

Add global average pooling and dense output

In [20]:

x = base_model.output
x = GlobalAveragePooling2D()(x)
output = Dense(10, activation='softmax')(x)
model = Model(inputs=base_model.input, outputs=output)

Compile and Train

In [21]:

from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, epochs=50, batch_size=64, validation_split=0.2, callbacks=callbacks)

Epoch 1/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 86s 62ms/step - accuracy: 0.5091 - loss: 1.6153 - val_accuracy: 0.1031 - val_loss: 13.2403 - learning_rate: 5.0000e-04
Epoch 2/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 33s 53ms/step - accuracy: 0.7518 - loss: 0.7633 - val_accuracy: 0.6239 - val_loss: 1.2840 - learning_rate: 5.0000e-04
Epoch 3/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 27s 42ms/step - accuracy: 0.7939 - loss: 0.6256 - val_accuracy: 0.5887 - val_loss: 1.4527 - learning_rate: 5.0000e-04
Epoch 4/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 37s 37ms/step - accuracy: 0.8078 - loss: 0.6046 - val_accuracy: 0.6129 - val_loss: 1.9547 - learning_rate: 5.0000e-04
Epoch 5/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 83s 104ms/step - accuracy: 0.8404 - loss: 0.4865 - val_accuracy: 0.8108 - val_loss: 0.5851 - learning_rate: 2.5000e-04
Epoch 6/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 25s 39ms/step - accuracy: 0.9315 - loss: 0.2011 - val_accuracy: 0.7938 - val_loss: 0.7051 - learning_rate: 2.5000e-04
Epoch 7/50
625/625 ━━━━━━━━━━━━━━━━━━━━ 39s 37ms/s

Evaluate

In [22]:

y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test_cat, axis=1)

accuracy = np.mean(y_pred_labels == y_true_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='macro')
recall = recall_score(y_true_labels, y_pred_labels, average='macro')

print(" ResNet50 on CIFAR-10:")
print(f" Accuracy:  {accuracy:.4f}")
print(f" Precision: {precision:.4f}")
print(f" Recall:    {recall:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 9s 18ms/step
 ResNet50 on CIFAR-10:
 Accuracy:  0.8328
 Precision: 0.8325
 Recall:    0.8328


#Section 3: Transformer Model – Training & Evaluation on CIFAR

In [23]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(factor=0.5, patience=2),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

In [24]:
import tensorflow as tf
from tensorflow.keras import layers
from sklearn.metrics import precision_score, recall_score
import numpy as np

Load and preprocess CIFAR-10

In [25]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

Parameters

In [26]:

input_shape = (32, 32, 3)
patch_size = 4
num_patches = (input_shape[0] // patch_size) ** 2
projection_dim = 64
num_heads = 4
transformer_units = [128, 64]
transformer_layers = 4
mlp_head_units = [128, 10]

Patch creation layer

In [27]:

class Patches(layers.Layer):
    def __init__(self, patch_size):
        super().__init__()
        self.patch_size = patch_size

    def call(self, images):
        batch_size = tf.shape(images)[0]
        patches = tf.image.extract_patches(
            images=images,
            sizes=[1, self.patch_size, self.patch_size, 1],
            strides=[1, self.patch_size, self.patch_size, 1],
            rates=[1, 1, 1, 1],
            padding="VALID",
        )
        patch_dims = patches.shape[-1]
        patches = tf.reshape(patches, [batch_size, -1, patch_dims])
        return patches

Positional encoding

In [28]:

class PatchEncoder(layers.Layer):
    def __init__(self, num_patches, projection_dim):
        super().__init__()
        self.num_patches = num_patches
        self.projection = layers.Dense(units=projection_dim)
        self.position_embedding = layers.Embedding(input_dim=num_patches, output_dim=projection_dim)

    def call(self, patch):
        positions = tf.range(start=0, limit=self.num_patches, delta=1)
        encoded = self.projection(patch) + self.position_embedding(positions)
        return encoded

 Build the Transformer model

In [29]:

def build_transformer_classifier():
    inputs = layers.Input(shape=input_shape)
    patches = Patches(patch_size)(inputs)
    encoded_patches = PatchEncoder(num_patches, projection_dim)(patches)

    for _ in range(transformer_layers):
        x1 = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
        attention_output = layers.MultiHeadAttention(num_heads=num_heads, key_dim=projection_dim)(x1, x1)
        x2 = layers.Add()([attention_output, encoded_patches])
        x3 = layers.LayerNormalization(epsilon=1e-6)(x2)
        x3 = layers.Dense(transformer_units[0], activation="relu")(x3)
        x3 = layers.Dense(transformer_units[1], activation="relu")(x3)
        encoded_patches = layers.Add()([x3, x2])

    representation = layers.LayerNormalization(epsilon=1e-6)(encoded_patches)
    representation = layers.Flatten()(representation)
    representation = layers.Dense(mlp_head_units[0], activation="relu")(representation)
    logits = layers.Dense(mlp_head_units[1], activation="softmax")(representation)

    # Final model
    model = tf.keras.Model(inputs=inputs, outputs=logits)
    return model


Compile & Train

In [30]:

model = build_transformer_classifier()
model.compile(optimizer="adam", loss="categorical_crossentropy", metrics=["accuracy"])
model.summary()
model.fit(x_train, y_train_cat, batch_size=64, epochs=25, validation_split=0.2)

Model: "functional_3"

┏━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)        ┃ Output Shape      ┃    Param # ┃ Connected to      ┃
┡━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━┩
│ input_layer_3       │ (None, 32, 32, 3) │          0 │ -                 │
│ (InputLayer)        │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patches (Patches)   │ (None, None, 48)  │          0 │ input_layer_3[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ patch_encoder       │ (None, 64, 64)    │      7,232 │ patches[0][0]     │
│ (PatchEncoder)      │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalization │ (None, 64, 64)    │        128 │ patch_encoder[0]… │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 64, 64)    │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add (Add)           │ (None, 64, 64)    │          0 │ multi_head_atten… │
│                     │                   │            │ patch_encoder[0]… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add[0][0]         │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_6 (Dense)     │ (None, 64, 128)   │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_7 (Dense)     │ (None, 64, 64)    │      8,256 │ dense_6[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_1 (Add)         │ (None, 64, 64)    │          0 │ dense_7[0][0],    │
│                     │                   │            │ add[0][0]         │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add_1[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ multi_head_attenti… │ (None, 64, 64)    │     66,368 │ layer_normalizat… │
│ (MultiHeadAttentio… │                   │            │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_2 (Add)         │ (None, 64, 64)    │          0 │ multi_head_atten… │
│                     │                   │            │ add_1[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add_2[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_8 (Dense)     │ (None, 64, 128)   │      8,320 │ layer_normalizat… │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ dense_9 (Dense)     │ (None, 64, 64)    │      8,256 │ dense_8[0][0]     │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ add_3 (Add)         │ (None, 64, 64)    │          0 │ dense_9[0][0],    │
│                     │                   │            │ add_2[0][0]       │
├─────────────────────┼───────────────────┼────────────┼───────────────────┤
│ layer_normalizatio… │ (None, 64, 64)    │        128 │ add_3[0][0]       │
│ (LayerNormalizatio… │                   │            │                   │
├─────────────────────┼───────────────────┼────────────┼─────────────────

 Total params: 865,866 (3.30 MB)

 Trainable params: 865,866 (3.30 MB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 29s 20ms/step - accuracy: 0.2015 - loss: 2.1892 - val_accuracy: 0.3714 - val_loss: 1.6951
Epoch 2/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 11s 16ms/step - accuracy: 0.4242 - loss: 1.5582 - val_accuracy: 0.4860 - val_loss: 1.4165
Epoch 3/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.5109 - loss: 1.3500 - val_accuracy: 0.5410 - val_loss: 1.2670
Epoch 4/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.5654 - loss: 1.2000 - val_accuracy: 0.5676 - val_loss: 1.1961
Epoch 5/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.6106 - loss: 1.0896 - val_accuracy: 0.5877 - val_loss: 1.1555
Epoch 6/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 16ms/step - accuracy: 0.6453 - loss: 1.0033 - val_accuracy: 0.6134 - val_loss: 1.0983
Epoch 7/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 9s 15ms/step - accuracy: 0.6682 - loss: 0.9369 - val_accuracy: 0.6096 - val_loss: 1.1012
Epoch 8/25
625/625 ━━━━━━━━━━━━━━━━━━━━ 10s 15ms/step - accuracy: 0.7030 - loss: 0.8418 - v

Evaluate

In [31]:

y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test_cat, axis=1)

accuracy = np.mean(y_pred_labels == y_true_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='macro')
recall = recall_score(y_true_labels, y_pred_labels, average='macro')

print(" Transformer (Custom) on CIFAR-10:")
print(f" Accuracy:  {accuracy:.4f}")
print(f" Precision: {precision:.4f}")
print(f" Recall:    {recall:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 4s 7ms/step
 Transformer (Custom) on CIFAR-10:
 Accuracy:  0.6246
 Precision: 0.6266
 Recall:    0.6246


#Section 4: VGG16 Model – Training & Evaluation on CIFAR

In [32]:
from tensorflow.keras.callbacks import ReduceLROnPlateau, ModelCheckpoint

callbacks = [
    ReduceLROnPlateau(factor=0.5, patience=2),
    ModelCheckpoint("best_model.keras", save_best_only=True)
]

In [33]:

import tensorflow as tf
from tensorflow.keras.applications import VGG16
from tensorflow.keras.models import Model
from tensorflow.keras.layers import GlobalAveragePooling2D, Dense
from sklearn.metrics import precision_score, recall_score
import numpy as np

Load and Preprocess CIFAR-10

In [34]:

(x_train, y_train), (x_test, y_test) = tf.keras.datasets.cifar10.load_data()
x_train, x_test = x_train / 255.0, x_test / 255.0
y_train_cat = tf.keras.utils.to_categorical(y_train, 10)
y_test_cat = tf.keras.utils.to_categorical(y_test, 10)

Load pre-trained VGG16 without top

In [35]:

base_model = VGG16(include_top=False, weights='imagenet', input_shape=(32, 32, 3))

58889256/58889256 ━━━━━━━━━━━━━━━━━━━━ 0s 0us/step


Freeze the convolutional base to save memory

In [36]:

base_model.trainable = False

Add custom classification head

In [37]:

x = base_model.output
x = GlobalAveragePooling2D()(x)          # Reduce dimensions safely
x = Dense(128, activation='relu')(x)
output = Dense(10, activation='softmax')(x)

model = Model(inputs=base_model.input, outputs=output)

Compile and Train

In [38]:

from tensorflow.keras.optimizers import Adam
model.compile(optimizer=Adam(learning_rate=0.0005), loss='categorical_crossentropy', metrics=['accuracy'])
model.fit(x_train, y_train_cat, epochs=25, batch_size=32, validation_split=0.2)

Epoch 1/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 19s 12ms/step - accuracy: 0.4126 - loss: 1.6825 - val_accuracy: 0.5447 - val_loss: 1.3108
Epoch 2/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 16s 10ms/step - accuracy: 0.5662 - loss: 1.2529 - val_accuracy: 0.5584 - val_loss: 1.2448
Epoch 3/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.5894 - loss: 1.1865 - val_accuracy: 0.5836 - val_loss: 1.1964
Epoch 4/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 12s 10ms/step - accuracy: 0.6047 - loss: 1.1435 - val_accuracy: 0.5941 - val_loss: 1.1730
Epoch 5/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.6154 - loss: 1.1132 - val_accuracy: 0.5973 - val_loss: 1.1700
Epoch 6/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.6165 - loss: 1.0942 - val_accuracy: 0.5976 - val_loss: 1.1585
Epoch 7/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 20s 10ms/step - accuracy: 0.6274 - loss: 1.0707 - val_accuracy: 0.6019 - val_loss: 1.1542
Epoch 8/25
1250/1250 ━━━━━━━━━━━━━━━━━━━━ 21s 10ms/step - accuracy: 0.6383 -

Evaluate

In [39]:

y_pred = model.predict(x_test)
y_pred_labels = np.argmax(y_pred, axis=1)
y_true_labels = np.argmax(y_test_cat, axis=1)

accuracy = np.mean(y_pred_labels == y_true_labels)
precision = precision_score(y_true_labels, y_pred_labels, average='macro')
recall = recall_score(y_true_labels, y_pred_labels, average='macro')

print(" VGG16 on CIFAR-10:")
print(f" Accuracy:  {accuracy:.4f}")
print(f" Precision: {precision:.4f}")
print(f" Recall:    {recall:.4f}")

313/313 ━━━━━━━━━━━━━━━━━━━━ 6s 18ms/step
 VGG16 on CIFAR-10:
 Accuracy:  0.6065
 Precision: 0.6124
 Recall:    0.6065
